Notebook by:

* Lorenzo Pannacci 1948926
* ADD YOUR NAME

## Startup

In [1]:
######################
# LIBRARIES DOWNLOAD #
######################

install_packages = False
if install_packages:
    %pip install numpy pandas

In [2]:
####################
# LIBRARIES IMPORT #
####################

import numpy as np
import pandas as pd

In [3]:
#####################
# GLOBAL PARAMETERS #
#####################

dataset_path = r"data/vodclickstream_uk_movies_03.csv"

## 1. Recommendation sytem 

Implementing a recommendation system is critical for businesses and digital platforms that want to thrive in today's competitive environment. These systems use data-driven personalization to tailor content, products, and services to individual user preferences. The latter improves user engagement, satisfaction, retention, and revenue through increased sales and cross-selling opportunities. In this section, you will attempt to implement a recommendation system by identifying similar users' preferences and recommending movies they watch to the study user. 

To be more specific, you will implement your version of the [**LSH algorithm**](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/), which will take as input the user's preferred genre of movies, find the most similar users to this user, and recommend the most watched movies by those who are more similar to the user. 

__Data__: The data you will be working with can be found [here](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies).

Looking at the data, you can see that there is data available for each user for the movies the user <ins>clicked on</ins>. Gather the __title and genre__ of the __maximum top 10 movies__ that each user clicked on regarding the __number of clicks__.

In [4]:
# TODO

### 1.2 Minhash Signatures

Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket. 

__Important note:__ You must write your minhash function from scratch.  You are not permitted to use any already implemented hash functions.  Read the class materials and, if necessary, conduct an internet search.  The description of hash functions in the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) may be helpful as a reference.

In [5]:
# TODO

### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at __most five movies__ to the user to watch based on the movies clicked by similar users. 

To recommend at most five movies given a user_id, use the following procedure: 

1. Identify the <ins>two most similar</ins> users to this user.
2. If these two users have any movies __in common__, recommend those movies based on the total number of clicks by these users.
3. If there are __no more common__ movies, try to propose the most clicked movies by the __most similar user first__, followed by the other user. 

__Note:__ At the end of the process, we expect to see at most five movies recommended to the user.

__Example:__ assume you've identified user __A__ and __B__ as the most similar users to a single user, and we have the following records on these users: 

- User A with 80% similarity
- User B with 50% similarity
  
|user|movie title|#clicks|
|---|---|---|
|A|Wild Child|20|
|A|Innocence|10|
|A|Coin Heist|2|
|B|Innocence|30|
|B|Coin Heist|15|
|B|Before I Fall|30|
|B|Beyond Skyline|8|
|B|The Amazing Spider-Man|5|

- __Recommended movies__ in order:
    - Innocence
    - Coin Heist
    - Wild Child
    - Before I Fall
    - Beyond Skyline

In [6]:
# TODO

## 2. Grouping Users together!

Now, we will deal with clustering algorithms that will provide groups of Netflix users that are similar among them.

To solve this task, you must accomplish the following stages:

### 2.1 Getting your data + feature engineering

1)  Access to the data found in [this dataset](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies)

2)  Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as *feature engineering*. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features **for each** user:

    a)  Favorite genre (i.e., the genre on which the user spent the most time)

    b)  Average click duration

    c)  Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)

    d)  Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?

    e)  Average time spent a day by the user (considering only the days he logs in)

So, in the end, you should have for each user_id five features.

3)  Consider at least 10 additional features that can be generated for each user_id (you can use chatGPT or other LLM tools for suggesting features to create). Describe each of them and add them to the previous dataset you made (the one with five features). In the end, you should have for each user at least 15 features (5 recommended + 10 suggested by you).

---

We start by exploring our data and convert it in a more usable form. The dataset is light, we don't have to worry about dividing it into chunks.

---

In [7]:
# load the dataset from the .json file to a pandas dataframe
original_dataset = pd.read_csv(dataset_path)
original_dataset

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...,...
671731,730504,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,730505,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,730506,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,730507,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


In [8]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    671736 non-null  int64  
 1   datetime      671736 non-null  object 
 2   duration      671736 non-null  float64
 3   title         671736 non-null  object 
 4   genres        671736 non-null  object 
 5   release_date  671736 non-null  object 
 6   movie_id      671736 non-null  object 
 7   user_id       671736 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 41.0+ MB


---

The first thing we notice is the presence of a column called `Unnamed: 0` that has an incremental value. We can discard it and uses our index that starts from 0:

---

In [9]:
original_dataset.drop('Unnamed: 0', axis = 1, inplace = True)
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


---

It will be useful to convert both the the `datetime` and the `release_date` columns from a string to actual datetime format. We notice that some entries of `release_date` have value `NOT AVAILABLE`, we will have to convert them to NaT (Not a Time).

---

In [10]:
original_dataset.datetime = pd.to_datetime(original_dataset.datetime, errors='coerce')
original_dataset.release_date = pd.to_datetime(original_dataset.release_date, errors='coerce')
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


In [11]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   datetime      671736 non-null  datetime64[ns]
 1   duration      671736 non-null  float64       
 2   title         671736 non-null  object        
 3   genres        671736 non-null  object        
 4   release_date  641432 non-null  datetime64[ns]
 5   movie_id      671736 non-null  object        
 6   user_id       671736 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 35.9+ MB


---

The Kaggle page of the dataset (https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies) give us informations about the `duration` column:

`'duration' shows how long it was (in seconds) until that user clicked on another URL. A watch time of zero seconds means they visited the page but instantly clicked away.`

It is already a float, we don't need to treat it any further.

We can check the consistency of the dataset by checking if the same movie (i.e. all the entries with same `movie_id`) have the same characteristics (i.e. same `title`, `genres` and `release date`):

---

In [12]:
# iterate trough every movie
for movie_id, group_data in original_dataset.groupby('movie_id'):
    # check if there is more than a title for the same movie
    if group_data.title.unique().shape[0] != 1:
        print("Movie", movie_id, "has more than a title!")

    # check if there is more than a genre list for the same movie
    if group_data.genres.unique().shape[0] != 1:
        print("Movie", movie_id, "has conflicting genres!")

    # check if there is more than a release date for the same movie
    if group_data.release_date.unique().shape[0] != 1:
        print("Movie", movie_id, "has conflicting release dates!")

---

Since the `genres` column is a string we have to convert it to a list.

---

In [13]:
original_dataset.genres = original_dataset.genres.apply(lambda row: row.split(','))
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","[Comedy, Drama, Romance]",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"[Fantasy, Horror, Mystery, Thriller]",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"[Action, Thriller]",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"[Action, Drama]",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"[Animation, Action, Adventure, Comedy, Fam...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,[Talk-Show],2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"[Animation, Action, Adventure, Family, Sci...",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"[Action, Adventure, Sci-Fi]",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"[Documentary, Music]",2019-02-08,52d49c515a,0b8163ea4b


---

We want to build a new dataset of user features extracted from the current one. We start by creating a dataframe containing only `user_id` and we populate it each time we build a new feature.

---

In [14]:
# empty user dataframe
users_dataset_base = pd.DataFrame(original_dataset.user_id.unique(), columns = ["user_id"])


'''
    The first feature we want to build is:
    **Favorite genre (i.e., the genre on which the user spent the most time)**
    We can exploit the `explode` method of Pandas to get from each entry a new entry for each genre in the list:
'''

# create exploded dataframe
exploded_dataset = original_dataset.explode("genres")[["user_id", "genres", "duration"]]

# add together all values with same user_id and same genre
exploded_dataset = exploded_dataset.groupby(["user_id", "genres"]).sum()

# get most watched genre
exploded_dataset = exploded_dataset.groupby("user_id").idxmax()
exploded_dataset.duration = exploded_dataset.duration.apply(lambda row: row[1])
exploded_dataset = exploded_dataset.rename(columns={'duration': 'favorite_genre'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(exploded_dataset, left_on = "user_id", right_index = True)

'''
    The second feature to extract is:
    **Average click duration**
    This is very fast as we have just to calculate the mean of the `duration` columns of every user:
'''

mean_dataset = original_dataset.groupby("user_id").duration.mean().reset_index()
mean_dataset = mean_dataset.rename(columns={'duration': 'mean_duration'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(mean_dataset)

'''
    The third feature feature is:
    **Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)**

    We will use as delimitations between the three time parts:
    - Morning: 6 to 14
    - Afternoon: 14 to 20
    - Night: 20 to 6
'''

hours = original_dataset[["user_id", "datetime", "duration"]].copy()
hours.datetime = hours.datetime.dt.hour
hours.datetime = hours.datetime.apply(lambda x: 'morning' if 6 <= x < 14 else 'afternoon' if 14 <= x < 20 else 'night')
hours = hours.groupby(["user_id", "datetime"]).sum().groupby("user_id").idxmax()
hours.duration = hours.duration.apply(lambda row: row[1])
hours = hours.rename(columns={'duration': 'favorite_time_day'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(hours, left_on = "user_id", right_index = True)

'''
    The fourth feature is:
    **Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?**
'''

# We have to convert all datetimes to only the year and count for each user how many have release year > 2010 and not

years = original_dataset[["user_id", "release_date"]].copy()
years.release_date = years.release_date.dt.year
years["new"] = years.release_date > 2010
years["old"] = years.release_date <= 2010

user_oldnew = years.groupby("user_id").old.sum() > years.groupby("user_id").new.sum()
user_oldnew.name = "is_oldmovies_lover"

# merge into user dataset
users_dataset_base = users_dataset_base.merge(user_oldnew, left_on = "user_id", right_index = True)

'''
    The fifth feature is:
    **Average time spent a day by the user (considering only the days he logs in)**
'''

# We can just take the mean_duration column and divide it by the number of different days the same user has in the dataset.

# we count how many different days every user has logged in
days = original_dataset[["user_id", "datetime"]].copy()
days["datetime"] = days["datetime"].dt.floor('D')
days = days.groupby("user_id").datetime.nunique()

# we count the total duration per user
sums = original_dataset.groupby("user_id").duration.sum()

# divide
average_per_day = sums / days
average_per_day.name = "duration_per_day"

# merge into user dataset
users_dataset_base = users_dataset_base.merge(average_per_day, left_on = "user_id", right_index = True)

In [15]:
users_dataset_base

,user_id,favorite_genre,mean_duration,favorite_time_day,is_oldmovies_lover,duration_per_day
0,1dea19f6fe,Drama,0.000000,night,True,0.000000
1,544dcbc510,Fantasy,1553.750000,afternoon,False,18645.000000
2,7cbcc791bf,Thriller,4218.666667,afternoon,False,8437.333333
3,ebf43c36b6,Action,1883.533333,afternoon,False,9417.666667
4,a57c992287,Drama,3684.538462,afternoon,False,7369.076923
...,...,...,...,...,...,...
161913,45414be0ec,Romance,0.000000,afternoon,False,0.000000
161914,783ec67e84,Drama,0.000000,afternoon,True,0.000000
161915,89c715f3a4,Adventure,-1.000000,afternoon,False,-1.000000
161916,9207e1499b,Crime,42592.000000,night,False,42592.000000


---

The additional features we propose are:

- `watchtime`: the total watchtime of a certain user. It's the sum of all the durations.
- `iterations`: the total number of iterations of a certain user. It's the number of rows related to that user.
- `active_days`: the number of days the user has been active on the site. 
- `n_movies`: the number of unique movies watched by the user
- `max_rewatches`: the maximum number of times the user has watched the same movie
- `favorite_day`: the user favourite day of the week to watch movies. We search it by finding the day with the highest duration overall for that user
- `favorite_month`: the user favourite month of the year to watch movies. We search it by finding the month with the highest duration overall for that user
- `most_active_year`: the year the user has been most active on based on the duration.
- `is_weekend_watcher`: if the user is a weekend watcher. We define a user "weekend watcher" if at least 50% of it's views are on saturday or sunday
- `longest_iteration`: the longest iteration registered for the user. We search it by finding the max across all durations

---

In [16]:
# create empty user dataframe
users_dataset_extended = pd.DataFrame(original_dataset.user_id.unique(), columns = ["user_id"])

# watchtime
n_movies = original_dataset.groupby("user_id").duration.sum()
n_movies.name = "total watchtime"

users_dataset_extended = users_dataset_extended.merge(n_movies, left_on = "user_id", right_index = True)

# iterations
n_iterations = original_dataset.groupby("user_id").size()
n_iterations.name = "iterations"

users_dataset_extended = users_dataset_extended.merge(n_iterations, left_on = "user_id", right_index = True)

# active_days
days_active = original_dataset[["user_id", "datetime"]].copy()
days_active["datetime"] = days_active["datetime"].dt.floor('D')
days_active = days_active.groupby("user_id").datetime.nunique()
days_active.name = "active_days"

users_dataset_extended = users_dataset_extended.merge(days_active, left_on = "user_id", right_index = True)

# n_movies
n_movies = original_dataset.groupby("user_id").movie_id.nunique()
n_movies.name = "n_movies"
users_dataset_extended = users_dataset_extended.merge(n_movies, left_on = "user_id", right_index = True)

# max_rewatches
rewatch = original_dataset.groupby(["user_id", "movie_id"]).size().groupby("user_id").max()
rewatch.name = "max_rewatches"

users_dataset_extended = users_dataset_extended.merge(rewatch, left_on = "user_id", right_index = True)

# favorite_day
favourite_day = original_dataset[["user_id", "datetime"]].copy()
favourite_day.datetime = favourite_day.datetime.dt.dayofweek
favourite_day = favourite_day.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
favourite_day.name = "favorite_day"

users_dataset_extended = users_dataset_extended.merge(favourite_day, left_on = "user_id", right_index = True)

# favorite_month
favourite_month = original_dataset[["user_id", "datetime"]].copy()
favourite_month.datetime = favourite_month.datetime.dt.month
favourite_month = favourite_month.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
favourite_month.name = "favorite_month"

users_dataset_extended = users_dataset_extended.merge(favourite_month, left_on = "user_id", right_index = True)

# most_active_year
most_active_year = original_dataset[["user_id", "datetime"]].copy()
most_active_year.datetime = most_active_year.datetime.dt.year
most_active_year = most_active_year.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
most_active_year.name = "most_active_year"

users_dataset_extended = users_dataset_extended.merge(most_active_year, left_on = "user_id", right_index = True)

# is_weekend_person
def weekend_function(row):
    row = row.droplevel(0)
    row = row.reindex(range(7), fill_value=0)
    return (row[5] + row[6]) > (row[0] + row[1] + row[2] + row[3] + row[4])

weekend = original_dataset[["user_id", "datetime"]].copy()
weekend.datetime = weekend.datetime.dt.dayofweek
weekend = weekend.groupby(["user_id", "datetime"]).size()
weekend = weekend.groupby("user_id").apply(weekend_function)
weekend.name = "is_weekend_user"

users_dataset_extended = users_dataset_extended.merge(weekend, left_on = "user_id", right_index = True)

# longest_iteration
longest_iteration = original_dataset.groupby("user_id").duration.max()
longest_iteration.name = "longest_iteration"

users_dataset_extended = users_dataset_extended.merge(longest_iteration, left_on = "user_id", right_index = True)

In [17]:
users_dataset_extended

,user_id,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
0,1dea19f6fe,0.0,1,1,1,1,6,1,2017,True,0.0
1,544dcbc510,18645.0,12,1,8,3,6,1,2017,True,10933.0
2,7cbcc791bf,25312.0,6,3,4,3,6,1,2017,False,10530.0
3,ebf43c36b6,28253.0,15,3,12,2,6,1,2017,True,7524.0
4,a57c992287,95798.0,26,13,22,3,0,1,2017,False,56338.0
...,...,...,...,...,...,...,...,...,...,...,...
161913,45414be0ec,0.0,1,1,1,1,6,6,2019,True,0.0
161914,783ec67e84,0.0,1,1,1,1,6,6,2019,True,0.0
161915,89c715f3a4,-1.0,1,1,1,1,6,6,2019,True,-1.0
161916,9207e1499b,42592.0,1,1,1,1,6,6,2019,True,42592.0


---

And finally we merge the two datasets.

---

In [18]:
users_dataset = users_dataset_base.merge(users_dataset_extended)
users_dataset

,user_id,favorite_genre,mean_duration,favorite_time_day,is_oldmovies_lover,duration_per_day,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
0,1dea19f6fe,Drama,0.000000,night,True,0.000000,0.0,1,1,1,1,6,1,2017,True,0.0
1,544dcbc510,Fantasy,1553.750000,afternoon,False,18645.000000,18645.0,12,1,8,3,6,1,2017,True,10933.0
2,7cbcc791bf,Thriller,4218.666667,afternoon,False,8437.333333,25312.0,6,3,4,3,6,1,2017,False,10530.0
3,ebf43c36b6,Action,1883.533333,afternoon,False,9417.666667,28253.0,15,3,12,2,6,1,2017,True,7524.0
4,a57c992287,Drama,3684.538462,afternoon,False,7369.076923,95798.0,26,13,22,3,0,1,2017,False,56338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161913,45414be0ec,Romance,0.000000,afternoon,False,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
161914,783ec67e84,Drama,0.000000,afternoon,True,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
161915,89c715f3a4,Adventure,-1.000000,afternoon,False,-1.000000,-1.0,1,1,1,1,6,6,2019,True,-1.0
161916,9207e1499b,Crime,42592.000000,night,False,42592.000000,42592.0,1,1,1,1,6,6,2019,True,42592.0


### 2.2 Choose your features (variables)!

You may notice that you have plenty of features to work with now. So, it would be best to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

1)  *To normalise or not to normalise? That's the question*. Sometimes, it is worth normalizing (scaling) the features. Explain if it is a good idea to perform any normalization method. If you think the normalization should be used, apply it to your data (look at the available normalization functions in the `scikit-learn` library).

2)  Select **one** method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

---

Two columns of our dataset, `favorite_genre` and `favorite_time_day` are strings and have to be converted in numerical values.

---

In [ ]:
# TODO

### 2.3 Clustering!

1)  Implement the K-means clustering algorithm (**not** ++: random initialization) using MapReduce. We ask you to write the algorithm from scratch following what you learned in class.

2)  Find an optimal number of clusters. Use at least two different methods. If your algorithms provide diverse optimal K's, select one of them and explain why you chose it.

3)  Run the algorithm on the data obtained from the dimensionality reduction.

4)  Implement **K-means++** from scratch and explain the differences with the results you got earlier.

5)  Ask ChatGPT to recommend other clustering algorithms and choose one. Explain your choice, then ask ChatGPT to implement it or use already implemented versions (e.g., the one provided in the scikit-learn library) and run it on your data. Explain the differences (if there are any) in the results. Which one is the best, in your opinion, and why?

In [ ]:
# TODO

### 2.4 Analysing your results!

You are often encouraged to explain the main characteristics that your clusters have. The latter is called the *Characterizing Clusters* step. Thus, follow the next steps to do it:

1)  Select 2-3 variables you think are relevant to identify the cluster of the customer. For example, Time_Day, Average Click Duration, etc.

2)  Most of your selected variables will be numerical (continuous or discrete), then categorize them into four categories.

3)  With the selected variables, perform pivot tables. On the horizontal axis, you will have the clusters, and on the vertical axis, you will have the categories of each variable. Notice that you have to do one pivot table per variable.

4)  Calculate the percentage by column for each pivot table. The sum of each row (cluster) must be 100.

5)  Interpret the results for each pivot table.

6)  Use any known metrics to estimate clustering algorithm performance (how good are the clusters you found?). Comment on the results obtained.

In [ ]:
# TODO

## 3. Bonus Question

We remind you that we consider and grade the bonuses only if you complete the entire assignment.

[Density-based clustering](https://wires.onlinelibrary.wiley.com/doi/epdf/10.1002/widm.30) identifies clusters as regions in the data space with high point density that are separated from other clusters by regions of low point density. The data points in the separating regions of low point density are typically considered noise or outliers. Typical algorithms that fall into this category are [OPTICS](https://dl.acm.org/doi/pdf/10.1145/304181.304187) and [DBSCAN](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf).

1)  Ask ChatGPT (or any other LLM tool) to list three algorithms for Density-Based Clustering. Choose one and use it on the same dataset you used in 2.3. Analyze your results: how different are they from the centroid-based version?

__Note__: You can implement your algorithm from scratch or use the one implemented in the scikit-learn library; the choice is up to you!

In [ ]:
# TODO

## 4. Command Line Question

Here is another command line question to enjoy. We previously stated that using the command line tools is a skill that Data Scientists must master.

In this question, you should use any command line tool that you know to answer the following questions using the same dataset that you have been using so far:
  + What is the most-watched Netflix title?
  + Report the average time between subsequent clicks on Netflix.com
  + Provide the ID of the user that has spent the most time on Netflix
    
__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.  

In [ ]:
# TODO

## 5. Algorithmic Question 

Federico studies in a demanding university where he has to take a certain number $N$ of exams to graduate,  but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system. 

He was given an initial personal score of $S$ when he enrolled, which changes every time he takes an exam: now comes the crazy part.
He soon discovered that every of the $N$ exams he has to take is assigned a mark $p$. Once he has chosen an exam, his score becomes equal to the mark $p$, and at the same time, the scoring system changes:
+ If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity $S - p$.
+ If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity $p - S$.
  
So, for example, consider $S=8$ as the initial personal score. Federico must decide which exam he wants to take, being $[5,7,1]$ the marks list. If he takes the first one, being $5 < 8$ and $8 - 5 = 3$, the remaining list now becomes $[10,4]$, and his score is updated as $S = 5$.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line,  and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get. 

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.

b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university! 

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

Here are some input/output examples (the first value is the initial personal score, and the second line contains the list of marks): 

__Input 1__
```
8
5 7 1 
```

__Output 1__
```
11
```

__Input 2__
```
25
18 24 21 32 27
```

__Output 2__
```
44
```

__Input 3__
```
30
13 27 41 59 28 33 39 19 52 48 55 79
```

__Output 3__
```
109
```

In [ ]:
# TODO